In [20]:
import yaml
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType, IntegerType, FloatType


In [21]:
def write_DB (parquet_DF, table):
    try:
        parquet_DF.write.jdbc(
            url=jdbc_url,
            table=table,
            mode="overwrite",
            properties=properties
        )
    except Exception as error:
        logger.error("Error ruta .yaml:" + str(error))

In [22]:

spark = (
    SparkSession.builder
    .appName("Spark")
    .config("spark.driver.extraJavaOptions", r'-Dlog4j.configurationFile=file:/home/illidan/proyecto_desde0/ETL/log4j.properties')\
    .config("spark.jars", "/home/illidan/proyecto_desde0/postgre/jars/postgresql-42.7.4.jar") \
    .getOrCreate()
)



spark.sparkContext.setLogLevel("INFO")

logger = spark._jvm.org.apache.log4j.LogManager.getLogger(__name__)

logger.info("Log de ejemplo guardado en archivo y consola.")

try:
    #reading the config file
    #geting file path into a dictionary
    with open("/home/illidan/proyecto_desde0/Config_file/Config.Yaml", "r") as file:
        config = yaml.safe_load(file)
except Exception as error:
    logger.error("Error ruta .yaml:" + str(error))

errores_detectados = []


15:21:38.722 [Thread-4] INFO  __main__ - Log de ejemplo guardado en archivo y consola.


In [23]:
#BD conection

try:
    jdbc_url = "jdbc:postgresql://localhost:5433/sparkdb_dev"
    properties = {
        "user": config["sparkdb_dev"]["user"],
        "password": config["sparkdb_dev"]["password"],
        "driver": "org.postgresql.Driver"
    }
except Exception as error:
    logger.error("Error conection BD:" + str(error))



In [24]:
#Read parquet file
try:
    read_avg_flight = config['Parquet_file']['avg_flight']
    read_airline_in_airport = config['Parquet_file']['airline_in_airport']
    read_flights_per_cancell = config['Parquet_file']['flights_per_cancell']
    read_airport_notin_thelist = config['Parquet_file']['airport_notin_thelist']
except Exception as error:
    logger.error("Error read parquet:" + str(error))


try:
    avg_flight = spark.read.parquet(read_avg_flight)
    airline_in_airport =  spark.read.parquet(read_airline_in_airport)
    flights_per_cancell = spark.read.parquet(read_flights_per_cancell)
    airport_notin_thelist = spark.read.parquet(read_airport_notin_thelist)
except Exception as error:
    logger.error("Error read parquet:" + str(error))

15:21:38.767 [Thread-4] INFO  org.apache.spark.sql.execution.datasources.InMemoryFileIndex - It took 1 ms to list leaf files for 1 paths.
15:21:38.811 [Thread-4] INFO  org.apache.spark.SparkContext - Starting job: parquet at NativeMethodAccessorImpl.java:0
15:21:38.813 [dag-scheduler-event-loop] INFO  org.apache.spark.scheduler.DAGScheduler - Got job 20 (parquet at NativeMethodAccessorImpl.java:0) with 1 output partitions
15:21:38.813 [dag-scheduler-event-loop] INFO  org.apache.spark.scheduler.DAGScheduler - Final stage: ResultStage 20 (parquet at NativeMethodAccessorImpl.java:0)
15:21:38.813 [dag-scheduler-event-loop] INFO  org.apache.spark.scheduler.DAGScheduler - Parents of final stage: List()
15:21:38.813 [dag-scheduler-event-loop] INFO  org.apache.spark.scheduler.DAGScheduler - Missing parents: List()
15:21:38.814 [dag-scheduler-event-loop] INFO  org.apache.spark.scheduler.DAGScheduler - Submitting ResultStage 20 (MapPartitionsRDD[73] at parquet at NativeMethodAccessorImpl.java:0)

In [25]:
#write parquet file into DB

write_DB(avg_flight, "avg_flight")
write_DB(airline_in_airport,"airline_in_airport")
write_DB(flights_per_cancell,"flights_cancellation")
write_DB(airport_notin_thelist,"airport_notin_thelist")




15:21:39.328 [Thread-4] INFO  org.apache.spark.sql.execution.datasources.FileSourceStrategy - Pushed Filters: 
15:21:39.329 [Thread-4] INFO  org.apache.spark.sql.execution.datasources.FileSourceStrategy - Post-Scan Filters: 
15:21:39.341 [Thread-4] INFO  org.apache.spark.storage.memory.MemoryStore - Block broadcast_32 stored as values in memory (estimated size 200.9 KiB, free 433.8 MiB)
15:21:39.358 [dispatcher-BlockManagerMaster] INFO  org.apache.spark.storage.BlockManagerInfo - Removed broadcast_30_piece0 on 172.23.57.81:34293 in memory (size: 37.2 KiB, free: 434.3 MiB)
15:21:39.365 [dispatcher-BlockManagerMaster] INFO  org.apache.spark.storage.BlockManagerInfo - Removed broadcast_31_piece0 on 172.23.57.81:34293 in memory (size: 37.2 KiB, free: 434.4 MiB)
15:21:39.366 [Thread-4] INFO  org.apache.spark.storage.memory.MemoryStore - Block broadcast_32_piece0 stored as bytes in memory (estimated size 34.9 KiB, free 434.0 MiB)
15:21:39.366 [dispatcher-BlockManagerMaster] INFO  org.apache.